In [1]:
import torch

In [2]:
x = torch.arange(6).T
x.view(2,3)

/tmp/ipykernel_21185/1642614056.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/aten/src/ATen/native/TensorShape.cpp:3683.)
  x = torch.arange(6).T


tensor([[0, 1, 2],
        [3, 4, 5]])

In [4]:
tensor2d = torch.tensor([[1,2,3],
                         [4,5,6]])

tensor2d.shape

torch.Size([2, 3])

In [5]:
tensor2d.T

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [6]:
tensor2d @ tensor2d.T

tensor([[14, 32],
        [32, 77]])

In [12]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])
z = x1*w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a,y)

In [13]:
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)
z = x1*w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a,y)

loss.backward()

print(loss)
print(w1.grad)
print(b.grad)

tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([-0.0898])
tensor([-0.0817])


In [57]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, inputs, outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # Output layer
            torch.nn.Linear(20, outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [58]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [59]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)

2213


In [60]:
50*30 + 30 + 30*20 + 20 + 20*3 + 3

2213

In [61]:
model.layers[0].weight

Parameter containing:
tensor([[-0.1273,  0.0692, -0.1022,  ...,  0.0763, -0.0978, -0.0687],
        [ 0.1153,  0.0354,  0.0517,  ..., -0.0764, -0.0498, -0.0598],
        [ 0.0314,  0.0854, -0.0648,  ..., -0.0587, -0.0399, -0.0110],
        ...,
        [-0.1169,  0.0513, -0.0310,  ...,  0.0441,  0.0058, -0.1277],
        [ 0.1324, -0.0696,  0.0440,  ..., -0.0368, -0.0736,  0.1113],
        [-0.1006, -0.1028,  0.0742,  ..., -0.0780,  0.0295,  0.0728]],
       requires_grad=True)

In [62]:
model.layers[0].weight.shape

torch.Size([30, 50])

In [63]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [64]:
from torch.utils.data import DataLoader, Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        one_X = self.X[index]
        one_y = self.y[index]
        return one_X, one_y
    
    def __len__(self):
        return self.y.shape[0]
    
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

print(len(train_ds))

5


In [65]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [66]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-0.5000,  2.6000],
        [-1.2000,  3.1000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [67]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [ 2.7000, -1.5000]]) tensor([0, 1])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


In [68]:
import torch.nn.functional as F

torch.manual_seed(123)

model = NeuralNetwork(inputs=2, outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()

    for idx, (X_batch, y_batch) in enumerate(train_loader):
        logits = model(X_batch)

        loss = F.cross_entropy(logits, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### Logging
        print(f"Epoch {epoch+1}, Batch {idx+1}, Loss: {loss.item()}")

    model.eval()

Epoch 1, Batch 1, Loss: 0.7487356662750244
Epoch 1, Batch 2, Loss: 0.6450306177139282
tensor([[ 1.0761, -1.9320],
        [ 0.9304, -1.7248],
        [ 0.7272, -1.4390],
        [-0.2722,  0.2662],
        [-0.3136,  0.3228]])
Epoch 2, Batch 1, Loss: 0.44225579500198364
Epoch 2, Batch 2, Loss: 0.12562280893325806
tensor([[ 2.7292, -4.0033],
        [ 2.4228, -3.6111],
        [ 1.9964, -3.0593],
        [-1.2539,  1.2540],
        [-1.4510,  1.4673]])
Epoch 3, Batch 1, Loss: 0.026905544102191925
Epoch 3, Batch 2, Loss: 0.0043272399343550205
tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [69]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)


In [72]:
2*30 + 30 + 30*20 + 20 + 20*2 + 2

752

In [71]:
sum([p.numel() for p in model.parameters() if p.requires_grad])

752

In [73]:
with torch.no_grad():
    outputs = model(X_train)
    print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [74]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [75]:
predictions = torch.argmax(probas, dim=1)
print(predictions == y_train)

tensor([True, True, True, True, True])


In [76]:
def compute_accuracy(model, dataloader):
    model = model.eval()

    num_correct = 0
    total_examples = 0

    for X_batch, y_batch in dataloader:
        logits = model(X_batch)
        predictions = torch.argmax(logits, dim=1)
        num_correct += torch.sum(predictions == y_batch)
        total_examples += y_batch.shape[0]

    return num_correct/total_examples

In [77]:
print(compute_accuracy(model, train_loader))

tensor(1.)


In [78]:
print(compute_accuracy(model, test_loader))

tensor(1.)


In [80]:
torch.save(model.state_dict(), 'model.pth')

In [81]:
model = NeuralNetwork(inputs=2, outputs=2)
model.load_state_dict(torch.load('model.pth'))

/tmp/ipykernel_21185/1583171545.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))


<All keys matched successfully>

In [82]:
print(torch.cuda.is_available())

True


In [83]:
tensor_1 = torch.tensor([1,2,3])
tensor_2 = torch.tensor([4,5,6])
print(tensor_1 + tensor_2)

tensor([5, 7, 9])


In [85]:
tensor_1.to('cuda') + tensor_2.to('cuda')

tensor([5, 7, 9], device='cuda:0')

In [87]:
import torch.nn.functional as F

model = NeuralNetwork(inputs=2, outputs=2)
device = ("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        logits = model(X_batch)
        loss = F.cross_entropy(logits, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### Logging
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    
    model.eval()

Epoch 1, Loss: 0.5388295650482178
Epoch 1, Loss: 0.22660142183303833
Epoch 2, Loss: 0.07637225091457367
Epoch 2, Loss: 0.01649988815188408
Epoch 3, Loss: 0.02446313202381134
Epoch 3, Loss: 0.027388697490096092
